# Setup prerequisites

In this brief tutorial, we'll make sure you get everything set up so you can run the other notebooks.

1. First, you'll need Visual Studio Code. You'll also need the Polyglot extension from Microsoft.
2. In order to make progress in these tutorials, you must have either Azure OpenAI Service provisioned and/or established an OpenAI account.
3. Find the **config** folder and add a new file there named **settings.json**.
4. In the settings.json file you just created, paste the following code and replace the text within square brackets with your own settings. (Remember you only to populate either AzureOpenAI or OpenAI settings.): 
``` json
{
    "Type": "azure", 
    "AzureOpenAI": {
        "Endpoint": "[the endpoint URL from your Azure OpenAI Service instance]", 
        "ApiKey": "[key from your Azure OpenAI Service instance.]", 
        "ChatDeployment": "[insert the name of a model deployment in your Azure OpenAI instance.]"
    }, 
    "OpenAI": {
        "ApiKey": "[the key from your OpenAI instance]", 
        "OrgId": "[the organization ID from your OpenAI instance]", 
        "Model": "[insert the name of the model you are using from OpenAI]"
    }
}
```
For the **type** setting above, you can enter *azure* to specify you want to use Azure OpenAI Service or *openai* if you want to use OpenAI.

If you are connecting to an instance of the Azure OpenAI Service, you have to set:
- **AzureOpenAI Endpoint** In the Azure portal, navigate to the **Overview** page of your Azure OpenAI instance. Copy the Endpoint URL to your clipboard and paste the value into your settings.json.
- **AzureOpenAIApiKey** In the Azure portal, navigate to the **Keys and Endpoint** page of your Azure OpenAI instance. Copy the **KEY 1** value to your clipboard and paste the value into your settings.json.
- **deployment** In the Azure portal, navigate to the **Model Deployments** page of your Azure OpenAI instance. Click the **Manage Deployments** button to navigate to Azure AI Studio and the **Deployments** page. If you haven't already, create a new deployment and select the gpt-35-turbo model. Give your deployment a name and paste the value into your settings.json. NOTE: The name of the deployment you set is the value to use in the settings.json - which may differ from the name of the underlying model.
Else if you are connecting directly to the OpenAI APIs, first navigate to [this](https://platform.openai.com/) and log in using your Microsoft account.
- **ApiKey** In your browser, navigate to [this](https://platform.openai.com/account/api-keys) web page. Copy one of your API keys from the web page to your clipboard and paste the value into your settings.json.
- **Organization** In your browser, navigate to [this](https://platform.openai.com/account/org-settings). Copy the organization ID from the web page to your clipboard and paste the value into your settings.json.
- **Model** In your browser, navigate to [this](https://platform.openai.com/docs/models). You can (and should) explore the various models available from OpenAI, but for most of these tutorials, I recommend starting with gpt-3.5-turbo. Copy your chosen model to your clipboard and paste the value into your settings.json.

In [ ]:
// hit the little play button to the left of this line in order to run this code
#!import ../config/SettingsHelper.cs

MySettings settings = Settings.LoadFromFile();

// print the settings from your file below
Console.WriteLine($"I want to use {settings.Type}");
Console.WriteLine($"Using Azure OpenAI endpoint: {settings.AzureOpenAI.Endpoint}");
Console.WriteLine($"Your key is secret, but we see {settings.AzureOpenAI.ApiKey.Length} characters in your setting.");
Console.WriteLine($"Using chat deployment: {settings.AzureOpenAI.ChatDeployment}");
Console.WriteLine($"Using completions deployment: {settings.AzureOpenAI.CompletionsDeployment}");
Console.WriteLine($"Your OpenAI API key is secret, but we see {settings.OpenAI.ApiKey.Length} characters in your setting.");
Console.WriteLine($"Using OpenAI Organization ID, but we see {settings.OpenAI.OrgId.Length} characters in your setting.");
Console.WriteLine($"Using OpenAI model: {settings.OpenAI.Model}");

If you see your settings correctly printed above, then you have set up your settings.json well. Great job!!

We still have to ensure you entered the correct settings and you are able to access your instance of Azure OpenAI Service or OpenAI's APIs. The rest of the tutorials in this series will follow the pattern below:

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.7.1"
#!import ../config/SettingsHelper.cs
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;

MySettings settings = Settings.LoadFromFile();
IKernelBuilder builder = Kernel.CreateBuilder();
if (settings.Type == "azure")
    builder.AddAzureOpenAIChatCompletion(
        settings.AzureOpenAI.ChatDeployment, "gpt-35-turbo", settings.AzureOpenAI.Endpoint, settings.AzureOpenAI.ApiKey);
else
    builder.AddOpenAIChatCompletion(
        settings.OpenAI.Model, settings.OpenAI.ApiKey, settings.OpenAI.OrgId);
Kernel kernel = builder.Build();

// define your semantic function
string functionDefinition = """
    {{$input}}...
    Finish the phrase above.
    """;

// optionally, you can specify settings for the function
OpenAIPromptExecutionSettings executionSettings = new() { MaxTokens = 25 };

// attach the function to your kernel
KernelFunction finishFunction = kernel.CreateFunctionFromPrompt(functionDefinition, executionSettings);

// invoke the function to pass the prompt as input to the function
FunctionResult result = await finishFunction.InvokeAsync(kernel, new KernelArguments() {{"input", "How much wood could"}});

// return the result to the user
Console.Write(result.GetValue<string>()!);

Did you get the output you were expecting? If you see the completed phrase, great job!! If you don't, then double-check:
- your seettings in settings.json and 
- be sure you correctly set your deployment (Azure) or model (OpenAI).

Don't worry too much about what each of the above lines of code actually does as we'll walk through this in more detail in the next few tutorials.